# quantify degree of geostrophy with instantaneous fields

Xiaolong initial [notebook](https://github.com/xy6g13/equinox_working/blob/master/sandbox/Reconstruction/Geostrophy%20metric.ipynb)

In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

import xgcm

from mitequinox.utils import *
from mitequinox.dynamic import *

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=1)
#print(cluster.job_script())
w = cluster.scale(30)

In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.135.39.90:37671 Dashboard: http://10.135.39.90:8787/status,Cluster Workers: 30 Cores: 30 Memory: 1.20 TB


---
# load data


In [6]:
#grd = load_grdnc().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs']).load()
grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])

dsU = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('SSU'))
dsV = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('SSV'))
dsE = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('Eta'))

ds = (xr.merge([dsU,dsV, dsE])
      .assign_coords(**grd.variables))
grid = xgcm.Grid(ds, periodic=['X', 'Y'])

# add coriolis parameters to dataset
#omega = 7.3/100000 # XY, 
# see, http://mitgcm.org/public/r2_manual/final/code_reference/vdb/names/R.htm
omega = 2.*np.pi/86164
f = 2*omega*np.sin(np.deg2rad(ds['YG'])) # at vorticity points
f_i = grid.interp(f,'X').chunk({'i':None}) # at v points
f_j = grid.interp(f,'Y').chunk({'j':None}) # at u points
ds0 = ds.assign_coords(f=f,f_i=f_i,f_j=f_j)


---

## start computing and storing

is the storing really necessary?

In [ ]:
#for face in [0,1]:
for face in range(13):

    ds = ds0.sel(face=face)
    # 
    for t in ['u_coriolis_linear','u_gradp', 'v_coriolis_linear','v_gradp']:
        bterm = get_mbal(t, ds, grid)
        # could subsample temporally and spatially here
        #print(bterm)
        file = work_data_dir+'mbal/%s_f%02d.zarr'%(t,face)
        if not os.path.isdir(file):
            bterm.to_dataset().to_zarr(work_data_dir+
                                       'mbal/%s_f%02d.zarr'% (t, face), mode='w')
        print('%s, face=%d - processed'%(t, face))

u_coriolis_linear, face=0 - processed
u_gradp, face=0 - processed
v_coriolis_linear, face=0 - processed
v_gradp, face=0 - processed
u_coriolis_linear, face=1 - processed
u_gradp, face=1 - processed
v_coriolis_linear, face=1 - processed
v_gradp, face=1 - processed
u_coriolis_linear, face=2 - processed
u_gradp, face=2 - processed
v_coriolis_linear, face=2 - processed
v_gradp, face=2 - processed
u_coriolis_linear, face=3 - processed
u_gradp, face=3 - processed
v_coriolis_linear, face=3 - processed
v_gradp, face=3 - processed
u_coriolis_linear, face=4 - processed
u_gradp, face=4 - processed
v_coriolis_linear, face=4 - processed
v_gradp, face=4 - processed
u_coriolis_linear, face=5 - processed
u_gradp, face=5 - processed
v_coriolis_linear, face=5 - processed
v_gradp, face=5 - processed
u_coriolis_linear, face=6 - processed
u_gradp, face=6 - processed
v_coriolis_linear, face=6 - processed
v_gradp, face=6 - processed
u_coriolis_linear, face=7 - processed
u_gradp, face=7 - processed


---

## try to compute variances without intermediate storing

In [9]:
for face in range(13):

    ds = ds0.sel(face=face)
    # 
    dxgSSV_j = grid.interp(ds.dxG * ds['SSV'],'Y') 
    dxgSSV_ji = grid.interp(dxgSSV_j,'X')    # SSV at (i_g,j)
    Coriolis_u_linear = ds.f_j * dxgSSV_ji /ds.dxC    #f*SSV
    Coriolis_u_linear = Coriolis_u_linear.rename('Coriolis_u_linear')
    Coriolis_u_linear = Coriolis_u_linear.chunk({'i_g':None,'j':None})
    # could subsample temporally and spatially here
    #print(Coriolis_u_linear)
    print('Variable size: %.1f GB' %(Coriolis_u_linear.nbytes / 1e9))
    # could drop variables here: ['dxC','dyG']
    Coriolis_u_linear.to_dataset().to_zarr(
        work_data_dir+'mbal/Coriolis_u_linear_f%02d.zarr'% (face), mode='w')
    print('face=%d - allready processed'%(face))


<xarray.DataArray 'Coriolis_u_linear' (j: 4320, i_g: 4320, time: 8785)>
dask.array<shape=(4320, 4320, 8785), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
    face     int64 0
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    dxC      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    dyG      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    f_j      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Variable size: 655.8 GB
face=0 - allready processed
<xarray.DataArray 'Coriolis_u_linear' (j: 4320, i_g: 4320, time: 8785)>
dask.array<shape=(4320, 4320, 8785), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
    face     int64 1
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * 

face=11 - allready processed
<xarray.DataArray 'Coriolis_u_linear' (j: 4320, i_g: 4320, time: 8785)>
dask.array<shape=(4320, 4320, 8785), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
    face     int64 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    dxC      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    dyG      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    f_j      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Variable size: 655.8 GB
face=12 - allready processed


In [7]:
face=1

#dsU = xr.open_zarr(work_data_dir+'rechunked/%s_f%02d.zarr'%('SSU',face))
#dsV = xr.open_zarr(work_data_dir+'rechunked/%s_f%02d.zarr'%('SSV',face))
#dsE = xr.open_zarr(work_data_dir+'rechunked/%s_f%02d.zarr'%('Eta',face))
#dsE = (dsE.sel(time=slice(dsU.time[0], dsU.time[-1]))
#       .chunk({'time':240})
#      )

dsU = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('SSU')).isel(face=face)
dsV = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('SSV')).isel(face=face)
dsE = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('Eta')).isel(face=face)

ds = (xr.merge([dsU,dsV, dsE])
      .assign_coords(**grd.isel(face=face).variables))
grid = xgcm.Grid(ds, periodic=['X', 'Y'])

# add coriolis parameters to dataset
#omega = 7.3/100000 # XY, 
# see, http://mitgcm.org/public/r2_manual/final/code_reference/vdb/names/R.htm
omega = 2.*np.pi/86164
f = 2*omega*np.sin(np.deg2rad(ds['YG'])) # at vorticity points
f_i = grid.interp(f,'X').chunk({'i':None}) # at v points
f_j = grid.interp(f,'Y').chunk({'j':None}) # at u points
ds = ds.assign_coords(f=f,f_i=f_i,f_j=f_j)

print(ds)


<xarray.Dataset>
Dimensions:  (i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 8785)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
    face     int64 1
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    iters    (time) int64 dask.array<shape=(8785,), chunksize=(1,)>
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    CS       (j, i) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    Depth    (j, i) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    PHrefC   (k) float32 dask.array<shape=(90,), chunksize=(90,)>
    PHrefF   (k_p1) float32 dask.array<shape=(91,), chunksize=(91,)>
  

In [8]:
dxgSSV_j = grid.interp(ds.dxG * ds['SSV'],'Y') 
dxgSSV_ji = grid.interp(dxgSSV_j,'X')    # SSV at (i_g,j)
Coriolis_u_linear = ds.f_j * dxgSSV_ji /ds.dxC    #f*SSV
Coriolis_u_linear = Coriolis_u_linear.rename('Coriolis_u_linear')
Coriolis_u_linear = Coriolis_u_linear.chunk({'i_g':None,'j':None})
# could subsample temporally and spatially here
print(Coriolis_u_linear)
print('Variable size: %.1f GB' %(Coriolis_u_linear.nbytes / 1e9))
# could drop variables here: ['dxC','dyG']
Coriolis_u_linear.to_dataset().to_zarr(
    work_data_dir+'mbal/Coriolis_u_linear_f%02d.zarr'% (face), mode='w')
#var = Coriolis_u_linear.var(dim='time')
#print(var)
#var.plot()
print('face=%d - allready processed'%(face))

<xarray.DataArray 'Coriolis_u_linear' (j: 4320, i_g: 4320, time: 8785)>
dask.array<shape=(4320, 4320, 8785), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
    face     int64 1
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    dxC      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    dyG      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    f_j      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Variable size: 655.8 GB
face=1 - allready processed


## from netcdf files

In [6]:
#suff='_t00002*'
suff='_t*'
#dsU = load_datanc('SSU', parallel=True, chunks={'face':1, 'j':480}, suff=suff)
#dsV = load_datanc('SSV', parallel=True, chunks={'face':1, 'j_g':480}, suff=suff)
#dsE = load_datanc('Eta', parallel=True, chunks={'face':1, 'j':480}, suff=suff)
dsU = load_datanc('SSU', parallel=True, chunks={'face':1}, suff=suff)
dsV = load_datanc('SSV', parallel=True, chunks={'face':1}, suff=suff)
dsE = load_datanc('Eta', parallel=True, chunks={'face':1}, suff=suff)
print(dsU,dsE)

KeyboardInterrupt: 

In [37]:
grd = load_grdnc().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])
ds = (xr.merge([dsU,dsV, dsE.sel(time=slice(dsU.time[0], dsU.time[-1]))])
      .assign_coords(**grd.variables))

In [38]:
grid = xgcm.Grid(ds, periodic=['X', 'Y'])
grid

<xgcm.Grid>
Z Axis (not periodic):
  * center   k --> left
  * left     k_l --> center
  * outer    k_p1 --> center
  * right    k_u --> center
X Axis (periodic):
  * center   i --> left
  * left     i_g --> center
Y Axis (periodic):
  * center   j --> left
  * left     j_g --> center

In [39]:
# add coriolis parameters to dataset
#omega = 7.3/100000 # XY, 
# see, http://mitgcm.org/public/r2_manual/final/code_reference/vdb/names/R.htm
omega = 2.*np.pi/86164
f = 2*omega*np.sin(np.deg2rad(ds['YG'])) # at vorticity points
f_i = grid.interp(f,'X').chunk({'i':None}) # at v points
f_j = grid.interp(f,'Y').chunk({'j':None}) # at u points
ds = ds.assign_coords(f=f,f_i=f_i,f_j=f_j)

print(ds)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 8785)
Coordinates:
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
    iters    (time) int64 228096 228240 228384 ... 1492704 1492848 1492992
    dtime    (time) datetime64[ns] 2011-11-15 2011-11-15T01:00:00 ... 2012-11-15
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    CS       (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    Depth    (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    PHrefC   (k) float32 ...
    PHrefF   (k_p1) float32 ..

### linear coriolis term

In [40]:
ds0=ds

In [41]:
#def get_mbal(term):
#    if term is 'coriolis_linear_u': ...       

In [43]:
#for face in range(ds['face'].size):
face=1

ds = ds0.isel(face=face)

dxgSSV_j = grid.interp(ds.dxG * ds['SSV'],'Y') 
dxgSSV_ji = grid.interp(dxgSSV_j,'X')    # SSV at (i_g,j)
Coriolis_u_linear = ds.f_j * dxgSSV_ji /ds.dxC    #f*SSV
Coriolis_u_linear = Coriolis_u_linear.rename('Coriolis_u_linear')
Coriolis_u_linear = Coriolis_u_linear.chunk({'i_g':None,'j':None})
# could subsample temporally and spatially
print(Coriolis_u_linear)
print('Variable size: %.1f GB' %(Coriolis_u_linear.nbytes / 1e9))
# could drop variables here: ['dxC','dyG']
Coriolis_u_linear.to_dataset().to_zarr(
    work_data_dir+'mbal/Coriolis_u_linear_f%02d.zarr'% (face), mode='w')
#var = Coriolis_u_linear.var(dim='time')
#print(var)
#var.plot()
print('face=%d - allready processed'%(face))

<xarray.DataArray 'Coriolis_u_linear' (j: 4320, i_g: 4320, time: 8785)>
dask.array<shape=(4320, 4320, 8785), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
    face     int64 1
    dxC      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    dyG      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    hFacW    (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    rAw      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    f_j      (j, i_g) float32 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 3.732e+07 3.732e+07
Variable size: 655.8 GB


distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)


KeyboardInterrupt: 

distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)


### all faces 

Goes through with 20 workers, 20 cores, 800GB, `chunks={'face':1, 'j':480}` and **1000 files** (i.e not the full dataset!).
Memory saturates at 515GB, approx 70% of 800GB (560GB truly), i.e. there is spilling to tmp disk. Wall time= 40min

In [9]:
out_dir = work_data_dir+'rechunked/'

for v in V:

    files = load_iters_date_files(v).file[:1000].tolist()
    #ds0 = load_datanc(v, files=files, parallel=True)
    #ds0 = load_datanc(v, parallel=True)
    ds0 = load_datanc(v, files=files, parallel=True, chunks={'face':1, 'j':480})
    
    Nt = len(ds.time) if Nt0 == 0 else Nt0
    
    #
    ds = ds0.isel(time=slice(len(ds0.time)//Nt *Nt))
    #
    chunks = {'time': Nt, 'i': Nc[0], 'j': Nc[1]}
    if v is 'SSU':
        chunks = {'time': Nt, 'i_g': Nc[0], 'j': Nc[1]}
    elif v is 'SSV':
        chunks = {'time': Nt, 'i': Nc[0], 'j_g': Nc[1]}
    ds = ds.chunk(chunks)

    file_out = out_dir+'%s.zarr'%(v)
    print(ds)
    try:
        %time ds.to_zarr(file_out, mode='w')
        # specify compression:
        #%time ds.to_zarr(file_out, mode='w', \
        #                 encoding={key: {'compressor': compressor} for key in ds.variables})
        # without compression: 601G for face 1
    except:
        print('Failure')
    dsize = getsize(file_out)
    print(' %s  data is %.1fGB ' %(v, dsize/1e9))


<xarray.Dataset>
Dimensions:  (face: 13, i_g: 4320, j: 4320, time: 960)
Coordinates:
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 ... 9.151e+06 9.155e+06
    iters    (time) int64 dask.array<shape=(960,), chunksize=(240,)>
    dtime    (time) datetime64[ns] dask.array<shape=(960,), chunksize=(240,)>
Data variables:
    SSU      (time, face, j, i_g) float32 dask.array<shape=(960, 13, 4320, 4320), chunksize=(240, 1, 48, 96)>


distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)


CPU times: user 27min 40s, sys: 1min 55s, total: 29min 35s
Wall time: 40min 36s
 SSU  data is 447.3GB 


---

In [17]:
w = cluster.scale_up(30)

In [ ]:
client.restart()

In [5]:
cluster.close()

In [2]:
xr.__version__

'0.11.0'